In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def list_files(iterate_dir="../results"):
    r = []
    for root, dirs, files in os.walk(iterate_dir):
        for name in files:
            filename = os.path.join(root, name)
            if "stats.csv" in filename and not "imagenet" in filename:
                r.append(filename)
    return r

In [3]:
stats_files = list_files()
print("Tenemos un total de {} stats files".format(len(stats_files)))

Tenemos un total de 5 stats files


### STATS

In [6]:
train_csv = pd.read_csv("../utils/data/train.csv")

In [7]:
def clean_stats(df, train_df):
    df = df.fillna(1)
    df["Vendor"] = "Z"
    df["Centre"] = 999
    df["Type"] = "XX"

    for i, row in df.iterrows():

        patient = row["patient"]
        c_phase = row["phase"]

        centre = train_df.loc[train_df["External code"]==patient].iloc[0]["Centre"]
        vendor = train_df.loc[train_df["External code"]==patient].iloc[0]["Vendor"]
        c_type = train_df.loc[(train_df["External code"]==patient) & (train_df["Phase"]==int(c_phase))].iloc[0]["Type"]

        df.at[i,'Vendor'] = vendor
        df.at[i,'Centre'] = centre
        df.at[i,'Type'] = c_type
    
    return df

In [10]:
print("Method \t\t\t\t\t IOU\t           DICE\t              Hausdorff          ASSD")
print("       \t\t\t\t\t A        B      | A        B      |  A        B      |  A        B      |")
print("-"*114)
table_global_mean_iou_a, table_global_mean_iou_b = [], []
table_global_mean_dice_a, table_global_mean_dice_b = [], []
table_global_mean_hausdorff_a, table_global_mean_hausdorff_b = [], []
table_global_mean_assd_a, table_global_mean_assd_b = [], []
for filename in stats_files:
    df = pd.read_csv(filename)
    df = clean_stats(df, train_csv)
    filename = filename[11:]
    model = filename.split("/")[1]
    data_augmentation = filename.split("/")[7][filename.split("/")[7].find("_da")+3:filename.split("/")[7].find("_scheduler")]
    optimizer = filename.split("/")[3]
    criterion = filename.split("/")[4][:filename.split("/")[4].find("_weights")]
    criterion_weights = filename.split("/")[4][filename.split("/")[4].find("_weights")+8:]
    lr = filename.split("/")[7][filename.split("/")[7].find("_lr")+3:]
    dataset = filename.split("/")[2]
    normalization = filename.split("/")[5][len("normalization_"):]
    train_vendor = filename.split("/")[7][8]
    test_vendor = filename.split("/")[7][18]
    
    mean_iou_lv = df[df["IOU_LV"] != -1].groupby("Vendor").mean()["IOU_LV"]
    mean_iou_rv = df[df["IOU_RV"] != -1].groupby("Vendor").mean()["IOU_RV"]
    mean_iou_myo = df[df["IOU_MYO"] != -1].groupby("Vendor").mean()["IOU_MYO"]
    global_mean_iou = (mean_iou_lv + mean_iou_rv + mean_iou_myo) / 3
    table_global_mean_iou_a.append(global_mean_iou["A"])
    table_global_mean_iou_b.append(global_mean_iou["B"])
    
    mean_dice_lv = df[df["DICE_LV"] != -1].groupby("Vendor").mean()["DICE_LV"]
    mean_dice_rv = df[df["DICE_RV"] != -1].groupby("Vendor").mean()["DICE_RV"]
    mean_dice_myo = df[df["DICE_LV"] != -1].groupby("Vendor").mean()["DICE_MYO"]
    global_mean_dice = (mean_dice_lv + mean_dice_rv + mean_dice_myo) / 3
    table_global_mean_dice_a.append(global_mean_dice["A"])
    table_global_mean_dice_b.append(global_mean_dice["B"])
    
    mean_hausdorff_lv = df[(df["HAUSDORFF_LV"] != -999) & (df["HAUSDORFF_LV"] != 999)].groupby("Vendor").mean()["HAUSDORFF_LV"]
    mean_hausdorff_rv = df[(df["HAUSDORFF_RV"] != -999) & (df["HAUSDORFF_RV"] != 999)].groupby("Vendor").mean()["HAUSDORFF_RV"]
    mean_hausdorff_myo = df[(df["HAUSDORFF_MYO"] != -999) & (df["HAUSDORFF_MYO"] != 999)].groupby("Vendor").mean()["HAUSDORFF_MYO"]
    global_mean_hausdorff = (mean_hausdorff_lv + mean_hausdorff_rv + mean_hausdorff_myo) / 3
    table_global_mean_hausdorff_a.append(global_mean_hausdorff["A"])
    table_global_mean_hausdorff_b.append(global_mean_hausdorff["B"])
    
    mean_assd_lv = df[(df["ASSD_LV"] != -999) & (df["ASSD_LV"] != 999)].groupby("Vendor").mean()["ASSD_LV"]
    mean_assd_rv = df[(df["ASSD_RV"] != -999) & (df["ASSD_RV"] != 999)].groupby("Vendor").mean()["ASSD_RV"]
    mean_assd_myo = df[(df["ASSD_MYO"] != -999) & (df["ASSD_MYO"] != 999)].groupby("Vendor").mean()["ASSD_MYO"]
    global_mean_assd = (mean_assd_lv + mean_assd_rv + mean_assd_myo) / 3
    table_global_mean_assd_a.append(global_mean_assd["A"])
    table_global_mean_assd_b.append(global_mean_assd["B"])
    
    method = "{} lr {} {}".format(data_augmentation[::2], lr, normalization)
    
    if True: # "none" == data_augmentation and lr =="0.005":
        print("{} fold {}\t {:.4f}   {:.4f} | {:.4f}   {:.4f} |  {:.4f}   {:.4f} |  {:.4f}   {:.4f} |".format(method, train_vendor, global_mean_iou["A"], global_mean_iou["B"], global_mean_dice["A"], global_mean_dice["B"], global_mean_hausdorff["A"], global_mean_hausdorff["B"], global_mean_assd["A"], global_mean_assd["B"]))

Method 					 IOU	           DICE	              Hausdorff          ASSD
       					 A        B      | A        B      |  A        B      |  A        B      |
------------------------------------------------------------------------------------------------------------------
cmiainod lr 0.001 standardize fold 3	 0.7458   0.7797 | 0.8222   0.8489 |  3.8657   3.3349 |  1.1234   0.9558 |
cmiainod lr 0.001 standardize fold 0	 0.7127   0.7659 | 0.7840   0.8411 |  4.2761   3.6185 |  1.2091   1.0008 |
cmiainod lr 0.001 standardize fold 1	 0.7557   0.7865 | 0.8295   0.8617 |  4.4001   3.6924 |  1.2161   1.0199 |
cmiainod lr 0.001 standardize fold 2	 0.7047   0.7884 | 0.7838   0.8612 |  5.5903   3.4184 |  1.5098   0.9581 |
cmiainod lr 0.001 standardize fold 4	 0.7475   0.7973 | 0.8241   0.8681 |  4.2544   3.3810 |  1.1710   0.8775 |


In [12]:
print(f"table_global_mean_iou_a {np.array(table_global_mean_iou_a).mean()}")
print(f"table_global_mean_iou_b {np.array(table_global_mean_iou_b).mean()}")

print(f"table_global_mean_dice_a {np.array(table_global_mean_dice_a).mean()}")
print(f"table_global_mean_dice_b {np.array(table_global_mean_dice_b).mean()}")

print(f"table_global_mean_hausdorff_a {np.array(table_global_mean_hausdorff_a).mean()}")
print(f"table_global_mean_hausdorff_b {np.array(table_global_mean_hausdorff_b).mean()}")

print(f"table_global_mean_assd_a {np.array(table_global_mean_assd_a).mean()}")
print(f"table_global_mean_assd_b {np.array(table_global_mean_assd_b).mean()}")

table_global_mean_iou_a 0.7332864777518415
table_global_mean_iou_b 0.7835851112018767
table_global_mean_dice_a 0.8087428730284859
table_global_mean_dice_b 0.8561972059204944
table_global_mean_hausdorff_a 4.477343513861242
table_global_mean_hausdorff_b 3.4890352110024963
table_global_mean_assd_a 1.2458696535304168
table_global_mean_assd_b 0.9624045796788476


In [ ]:
table_global_mean_iou_b

In [ ]:
table_global_mean_dice_a, table_global_mean_dice_b = [], []
table_global_mean_hausdorff_a, table_global_mean_hausdorff_b = [], []
table_global_mean_assd_a, table_global_mean_assd_b = [], []

In [9]:
fold_mean_str = "0.7458 | 0.7127 | 0.7557 | 0.7047 | 0.7475"
print("{:.4f}".format(np.array([float(val) for val in fold_mean_str.split("|")]).mean()))

0.7333
